In [ ]:
#coding:utf-8
from __future__ import division
from __future__ import print_function
import numpy as np
import tensorflow as tf
import gym
#gym环境
env=gym.make('CartPole-v0')

In [ ]:
def discount_rewards(r):
    """
    输入：
    1维的float类型数组，表示每个时刻的奖励
    输出：
    计算折扣率gamma后的期望奖励
    """
    discounted_r=np.zeros_like(r)
    running_add=0
    for t in reversed(range(0,r.size)):
        running_add=running_add*gamma+r[t]
        discounted_r[t]=running_add
    return discounted_r

In [ ]:
#参数设置
D=4 #输入层的维度
H=10 #隐藏层的维度
batch_size=5 #一个batch中有5个episode，即5次游戏
learning_rate=1e-2 
gamma=0.99 #奖励折扣率

#定义policy网络
# 输入观察值，输出右移的概率
observations=tf.placeholder(tf.float32,[None,D],name='input_x')
W1=tf.get_variable("W1",shape=[D,H],initializer=tf.contrib.layers.xavier_initializer())
layer1=tf.nn.relu(tf.matmul(observations,W1))
W2=tf.get_variable('W2',shape=[H,1],initializer=tf.contrib.layers.xavier_initializer())
score=tf.matmul(layer1,W2)
probability=tf.nn.sigmoid(score)

#定义相关的变量
tvars=tf.trainable_variables() #tf.trainable_variables返回的是需要训练的变量列表
input_y=tf.placeholder(tf.float32,[None,1],name='input_y')
advantages=tf.placeholder(tf.float32,name='reward_signal')

#定义loss函数
loglik=tf.log(input_y*(input_y-probability)+(1-input_y)*(input_y+probability))
loss = -tf.reduce_mean(loglik * advantages)
newGrads=tf.gradients(loss,tvars)

#优化器，梯度
adam=tf.train.AdamOptimizer(learning_rate=learning_rate)
W1Grad=tf.placeholder(tf.float32,name='batch_grad1')
W2Grad=tf.placeholder(tf.float32,name='batch_grad2')
batchGrad=[W1Grad,W2Grad]
updateGrads=adam.apply_gradients(zip(batchGrad,tvars))





xs,hs,dlogps,drs,ys,tfps=[],[],[],[],[],[]
running_reward=None
reward_sum=0
episode_number=1
total_episodes=10000
init=tf.global_variables_initializer()
with tf.Session() as sess:
    rendering=False
    sess.run(init)
    observation=env.reset() #observation是环境的初始观察量(输入神经网络的值)
    gradBuffer=sess.run(tvars) #gradBuffer会存储梯度，此处做一初始化
    for ix,grad in enumerate(gradBuffer):
        gradBuffer[ix]=grad*0
    while episode_number<=total_episodes:
        if reward_sum/batch_size>180 or rendering is True:
            env.render()
            rendering=True
        x=np.reshape(observation,[1,D]) #输入神经网络的值
        # action=1 表示向右移，action=0 表示向左移动，tfprob 网络输出向右走的概率
        tfprob=sess.run(probability,feed_dict={observations:x})
        # np.random.uniform()为0到1之间的随机数，当它小于tfprob时，采用右移策略，反之左移
        action = 1 if np.random.uniform()<tfprob else 0
        #xs记录每一步的观察量，ys记录每一步采取的策略
        xs.append(x)
        y=1 if action==0 else 0
        ys.append(y)
        #执行action
        observation,reward,done,info=env.step(action)
        reward_sum+=reward
        #drs记录每一步的reward
        drs.append(reward)
        #一局游戏结束
        if done:
            episode_number+=1
            #将xs，ys，drs从list变成numpy数组形式
            epx=np.vstack(xs)
            epy=np.vstack(ys)
            epr=np.vstack(drs)
            tfp=tfps
            xs,hs,dlogps,drs,ys,tfps=[], [], [], [], [], []  # reset
            discounted_epr=discount_rewards(epr) #对epr计算期望奖励
            # 对期望奖励做归一化
            discounted_epr-=np.mean(discounted_epr)
            discounted_epr //=np.std(discounted_epr)
            
            #将梯度保存到gradBuffer中
            tGrad=sess.run(newGrads,feed_dict={observations:epx,input_y:epy,advantages:discounted_epr})
            for ix,grad in enumerate(tGrad):
                gradBuffer[ix]+=grad
            #每batch_size局游戏，就将gradBuffer中的梯度真正更新到policy网络中
            if episode_number%batch_size==0:
                sess.run(updateGrads,feed_dict={W1Grad:gradBuffer[0],W2Grad:gradBuffer[1]})
                for ix,grad in enumerate(gradBuffer):
                    gradBuffer[ix]=grad*0
                print('Episode:%d~%d Average reward: %f.'%(episode_number-batch_size+1,
                                                          episode_number,reward_sum//batch_size))
                # 当平均分达到200分，停止训练
                if reward_sum//batch_size>=200:
                    print('Task solved in',episode_number,'episodes!')
                    break
                reward_sum=0
            observation=env.reset()
print(episode_number,'Episodes completed.')